In [ ]:
import numpy as np
import pandas as pd
import re
import ast
import pycountry
import requests
from tqdm import tqdm
import os

In [ ]:
# Prep functions
def evaluate_literal(val):
    try:
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        return val
    
def load(path):
    df = pd.read_csv(path)
    for col in df.columns:
        df[col] = df[col].apply(evaluate_literal)
    return df

def save(df):
    final_df = df[df.apply(is_row_valid, axis=1)]
    final_df = clean_df(final_df)
    try:
        temp_df = load('data/contract_elaborated.csv')
        temp_df = clean_df(temp_df)
        final_df = pd.concat([final_df, temp_df]).drop_duplicates(subset=['symbol', 'exact_search', 'search_exchange', 'search_symbol'])
    except FileNotFoundError:
        pass

    # Filter out the duplicates with 'exact_search' is False
    duplicates_df = final_df[final_df.duplicated(subset='symbol', keep=False)]
    final_df = final_df.drop(duplicates_df[duplicates_df['exact_search'] == False].index)

    final_df.to_csv('data/contract_elaborated.csv', index=False)

def is_numerical(val):
    try:
        val = str(val).replace('%', '')
        float(val)
        return True
    except Exception:
        return False

def is_valid_tuple(tuple, column):
    def extract_float(value):
        match = re.match(r'[^0-9]*([0-9.,]+)', value)
        if match:
            return float(match.group(1).replace(',', ''))
        return None
    
    label, value = tuple
    if not isinstance(label, str): # keep
        # if label != None: # Comment out for more rigid filter
        return False
    if value is None:
        return True # Comment out for more rigid filter
        return False 
    if is_numerical(value):
        return True
    
    if column == 'profile':
        # if value and label:
        return True
    if column == 'fundamentals':
        if value.isupper():
            return True
    if column == 'dividends':
        if value == 'Unknown':
            return True
        extract_float_value = extract_float(value)
        if extract_float_value is not None:
            return True
    if column == 'style':
        if isinstance(value, bool):
            return True
    return False

def is_row_valid(row):
    for col in row.index:
        if isinstance(row[col], list):
            # if col == 'fundamentals':
            #     if len(row[col]) not in [4,5,21,22,   23]: #4, 5, 21, 22 are the acceptable num of fund values, 23 is for little bugs
            #         print(len(row[col]))
            #         return False
            for tuple in row[col]:
                if not is_valid_tuple(tuple, col):
                    print(tuple)
                    return False
    return True

def has_bad_multiplier(long_name):
    cleaned = long_name.replace('-', '').replace('+', '')
    for word in cleaned.split():
        if re.fullmatch(r'\d+X', word):
            if int(word[:-1]) > 1:
                return True
    return False

def get_remaining():
    contract_details = load('data/contract_details.csv')
    try:
        final_df = load('data/contract_elaborated.csv')
        final_df = final_df[final_df.apply(is_row_valid, axis=1)]

        exclusion_condition = (final_df['exchange_bug'] == True) | (final_df['exact_search'] == True) | (~final_df['profile'].isna())
        # exclusion_condition = (final_df['exchange_bug'] == True) | (final_df['exact_search'] == True)
        symbols_to_exclude = final_df[exclusion_condition]['symbol']
        remaining = contract_details[~contract_details['symbol'].isin(symbols_to_exclude)]

        # # To debug invalid rows
        # remaining = final_df.copy()
        # remaining = remaining[~remaining.apply(is_row_valid, axis=1)]
    except FileNotFoundError:
        remaining = contract_details.copy()
        
    remaining = remaining[~remaining['longName'].apply(has_bad_multiplier)]
    remaining = remaining[['symbol', 'exchange', 'primaryExchange', 'validExchanges', 'currency', 'conId', 'longName', 'stockType', 'isin']]
    return remaining

In [ ]:
# Prep functions 2
def evaluate_literal(val):
    try:
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        return val
    
def load(path):
    df = pd.read_csv(path)
    for col in df.columns:
        df[col] = df[col].apply(evaluate_literal)
    return df

def save(df):
    final_df = df[df.apply(is_row_valid, axis=1)]
    final_df = clean_df(final_df)
    try:
        temp_df = load('data/contract_elaborated.csv')
        temp_df = clean_df(temp_df)
        final_df = pd.concat([final_df, temp_df]).drop_duplicates(subset=['symbol', 'exact_search', 'search_exchange', 'search_symbol'])
    except FileNotFoundError:
        pass

    # Filter out the duplicates with 'exact_search' is False
    duplicates_df = final_df[final_df.duplicated(subset='symbol', keep=False)]
    final_df = final_df.drop(duplicates_df[duplicates_df['exact_search'] == False].index)

    final_df.to_csv('data/contract_elaborated.csv', index=False)

def is_numerical(val):
    try:
        val = str(val).replace('%', '')
        float(val)
        return True
    except Exception:
        return False

def is_valid_tuple(tuple, column):
    def extract_float(value):
        match = re.match(r'[^0-9]*([0-9.,]+)', value)
        if match:
            return float(match.group(1).replace(',', ''))
        return None
    
    label, value = tuple
    if not isinstance(label, str): # keep
        # if label != None: # Comment out for more rigid filter
        return False
    if value is None:
        return True # Comment out for more rigid filter
        return False 
    if is_numerical(value):
        return True
    
    if column == 'profile':
        # if value and label:
        return True
    if column == 'fundamentals':
        if value.isupper():
            return True
    if column == 'dividends':
        if value == 'Unknown':
            return True
        extract_float_value = extract_float(value)
        if extract_float_value is not None:
            return True
    if column == 'style':
        if isinstance(value, bool):
            return True
    return False

def is_row_valid(row):
    for col in row.index:
        if isinstance(row[col], list):
            # if col == 'fundamentals':
            #     if len(row[col]) not in [4,5,21,22,   23]: #4, 5, 21, 22 are the acceptable num of fund values, 23 is for little bugs
            #         print(len(row[col]))
            #         return False
            for tuple in row[col]:
                if not is_valid_tuple(tuple, col):
                    print(tuple)
                    return False
    return True

def has_bad_multiplier(long_name):
    cleaned = long_name.replace('-', '').replace('+', '')
    for word in cleaned.split():
        if re.fullmatch(r'\d+X', word):
            if int(word[:-1]) > 1:
                return True
    return False

def get_remaining():
    contract_details = load('data/contract_details.csv')
    try:
        final_df = load('data/contract_elaborated.csv')
        final_df = final_df[final_df.apply(is_row_valid, axis=1)]

        exclusion_condition = (final_df['exchange_bug'] == True) | (final_df['exact_search'] == True) | (~final_df['profile'].isna())
        # exclusion_condition = (final_df['exchange_bug'] == True) | (final_df['exact_search'] == True)
        symbols_to_exclude = final_df[exclusion_condition]['symbol']
        remaining = contract_details[~contract_details['symbol'].isin(symbols_to_exclude)]

        # # To debug invalid rows
        # remaining = final_df.copy()
        # remaining = remaining[~remaining.apply(is_row_valid, axis=1)]
    except FileNotFoundError:
        remaining = contract_details.copy()
        
    remaining = remaining[~remaining['longName'].apply(has_bad_multiplier)]
    remaining = remaining[['symbol', 'exchange', 'primaryExchange', 'validExchanges', 'currency', 'conId', 'longName', 'stockType', 'isin']]
    return remaining

In [ ]:
# Cleaning functions
def clean_labels(label, col):
    if col == 'industries':
        if isinstance(label, str):
            if label.endswith('-Discontinuedeff09/19/2020'):
                return label.split('-')[0]
        return label
    
    elif col == 'holding_types':
        if isinstance(label, str):
            if label.startswith('■'):
                return label[1:]
            elif label.startswith('1'):
                return label[1:]
        return label
    elif col == 'debtors':
        if isinstance(label, str):
            if ('（') in label:
                return label.replace('（', '(')
        return label
    elif col == 'fundamentals':
        if isinstance(label, str):
            if label == 'LTDebt/ShareholdersEquity':
                return 'LTDebt/Shareholders'
        return label
    return label
    
def correct_digit(value_str):
    try:
        digit = re.sub(r'[^\d.-]', '', value_str).strip()
        return float(digit)
    except Exception:
        return value_str

def clean_values(value_str, col):
    # print(value_str)
    if col == 'profile':
        return value_str
    if isinstance(value_str, str):
        if value_str.endswith('%'):
            return correct_digit(value_str.replace('%',''))/100
        try:
            return correct_digit(value_str)
        except Exception:
            return value_str
    return value_str

def clean_df(df):
    for col in df.columns:
        # print(col)
        df[col] = df[col].apply(evaluate_literal)
        df[col] = df[col].apply(lambda x: [(clean_labels(item[0], col), item[1]) if isinstance(item, tuple) and len(item) == 2 else item for item in x] if isinstance(x, list) else x)
        df[col] = df[col].apply(lambda x: [(item[0], clean_values(item[1], col)) if isinstance(item, tuple) and len(item) == 2 else item for item in x] if isinstance(x, list) else x)
        df[col] = df[col].apply(lambda x: sorted(x, key=lambda item: item[0] if isinstance(item, tuple) and item[0] else '') if isinstance(x, list) else x)
    return df

In [ ]:
# Cleaning functions 2
def clean_labels(label, col):
    if col == 'industries':
        if isinstance(label, str):
            if label.endswith('-Discontinuedeff09/19/2020'):
                return label.split('-')[0]
        return label
    
    elif col == 'holding_types':
        if isinstance(label, str):
            if label.startswith('■'):
                return label[1:]
            elif label.startswith('1'):
                return label[1:]
        return label
    elif col == 'debtors':
        if isinstance(label, str):
            if ('（') in label:
                return label.replace('（', '(')
        return label
    elif col == 'fundamentals':
        if isinstance(label, str):
            if label == 'LTDebt/ShareholdersEquity':
                return 'LTDebt/Shareholders'
        return label
    return label
    
def correct_digit(value_str):
    try:
        digit = re.sub(r'[^\d.-]', '', value_str).strip()
        return float(digit)
    except Exception:
        return value_str

def clean_values(value_str, col):
    # print(value_str)
    if col == 'profile':
        return value_str
    if isinstance(value_str, str):
        if value_str.endswith('%'):
            return correct_digit(value_str.replace('%',''))/100
        try:
            return correct_digit(value_str)
        except Exception:
            return value_str
    return value_str

def clean_df(df):
    for col in df.columns:
        # print(col)
        df[col] = df[col].apply(evaluate_literal)
        df[col] = df[col].apply(lambda x: [(clean_labels(item[0], col), item[1]) if isinstance(item, tuple) and len(item) == 2 else item for item in x] if isinstance(x, list) else x)
        df[col] = df[col].apply(lambda x: [(item[0], clean_values(item[1], col)) if isinstance(item, tuple) and len(item) == 2 else item for item in x] if isinstance(x, list) else x)
        df[col] = df[col].apply(lambda x: sorted(x, key=lambda item: item[0] if isinstance(item, tuple) and item[0] else '') if isinstance(x, list) else x)
    return df

In [ ]:
# Explode columns
contracts_df = load('data/contract_elaborated.csv')
contracts_df = clean_df(contracts_df)
contracts_df = contracts_df[contracts_df.apply(is_row_valid, axis=1)]

contracts_df['bond'] = contracts_df[['debtors', 'maturity', 'debt_type']].notna().any(axis=1).astype(bool)
# contracts_df['bond'].replace(0.0, np.nan, inplace=True)
contracts_df[['equity', 'other']] = False

empty_subcategories = {
'holding_types': ['Other'],
'countries': ['Unidentified'], 
'currencies': ['<NoCurrency>'],
'industries': ['NonClassifiedEquity', 'NotClassified-NonEquity'],
'top10': ['AccountsPayable','AccountsReceivable','AccountsReceivable&Pay','AdministrationFees','CustodyFees','ManagementFees','OtherAssets','OtherAssetsandLiabilities','OtherAssetslessLiabilities',
          'OtherFees','OtherLiabilities','Tax','Tax--ManagementFees'],
'debtors': ['OTHER'],
'debt_type': ['%QualityNotAvailable', '%QualityNotRated'],
'maturity': ['%MaturityOther'],
}

original_columns = contracts_df.columns
columns_to_explode = ['profile', 'holding_types', 'style', 'lipper', 'fundamentals', 'countries', 'currencies', 'industries', 'top10', 'debtors', 'maturity', 'debt_type', 'dividends']
percentage_columns = [col for col in ['holding_types', 'countries', 'currencies', 'industries', 'top10', 'debtors', 'maturity', 'debt_type'] if col in columns_to_explode]
for col in columns_to_explode:
    # print(col)
    contracts_df[col] = contracts_df[col].fillna('[]')
    contracts_df[col] = contracts_df[col].apply(evaluate_literal)

    # Explode and create pivot_df
    contracts_df = contracts_df.explode(col)
    contracts_df[col] = contracts_df[col].apply(lambda x: (None, None) if pd.isna(x) else x)
    contracts_df[['label', 'value']] = pd.DataFrame(contracts_df[col].tolist(), index=contracts_df.index)

    pivot_df = contracts_df.pivot_table(index=contracts_df.index, columns='label', values='value', aggfunc='first')
    pivot_df.rename(columns={label: f'{col}_{label}' for label in pivot_df.columns}, inplace=True)

    # Correct pivot_vf values
    if col in percentage_columns:
        pivot_df = pivot_df.fillna(0.0).clip(lower=0)
        columns_to_drop = [f'{col}_{label}' for label in empty_subcategories[col]]
        pivot_cols = [pivot_col for pivot_col in pivot_df.columns if pivot_col not in columns_to_drop]

        # Scale error values so all sum to 1
        pivot_cols_sum = pivot_df.sum(axis=1)
        mask = pivot_cols_sum > 1
        pivot_df.loc[mask] = pivot_df.loc[mask].div(pivot_cols_sum[mask], axis=0)

        # Create variety columns
        pivot_df[f'{col}_variety'] = pivot_df.pow(2).sum(axis=1)
        pivot_df[f'{col}_variety'] = pivot_df[f'{col}_variety'].astype(float).replace(0.0, np.nan)

        # # To avoid multicollinearity
        # pivot_df = pivot_df.drop(columns=columns_to_drop, axis=1, errors='ignore')

        # Drop top10 company columns
        if col == 'top10':
            columns_to_drop = [column for column in pivot_df.columns if column != f'{col}_variety']
            pivot_df = pivot_df.drop(columns=columns_to_drop, axis=1)

    contracts_df = contracts_df.drop(columns=[col, 'label', 'value'], axis=1).drop_duplicates(subset='conId')
    contracts_df = pd.concat([contracts_df, pivot_df], axis=1)

contracts_df = contracts_df[~contracts_df['profile_TotalNetAssets'].isna()]

# # Fill NaN values with 0.0 for percentage columns
# for col in percentage_columns + [col for col in contracts_df.columns if col.startswith('dividends_')]:
#     full_columns = [full_column for full_column in contracts_df.columns if full_column.startswith(col) and full_column != f'{col}_variety']
#     contracts_df[full_columns] = contracts_df[full_columns].fillna(0.0)

In [ ]:
# Drop ETF duplicates
eur_exchanges = contracts_df[contracts_df['currency'] == 'EUR'].primaryExchange.unique()
remaining_columns = [col for col in contracts_df.columns if col not in original_columns]
og_len = len(contracts_df)

contracts_df = (
    contracts_df
    .assign(currency_is_euro=contracts_df['currency'] == 'EUR')
    .assign(exchange_is_european=contracts_df['exchange'].isin(eur_exchanges))
    .assign(primary_is_european=contracts_df['primaryExchange'].isin(eur_exchanges))
    .sort_values(by=['currency_is_euro','exchange_is_european', 'primary_is_european', 'tradable'], ascending=[False, False, False, False])
    .drop_duplicates(subset=remaining_columns, keep='first')
    .drop(columns=['currency_is_euro', 'exchange_is_european', 'primary_is_european'])
)
og_len - len(contracts_df)

In [ ]:
# Correct profile total net assets and TER
symbol_mapping = {
    '$': 'USD',    # Default to USD
    '￥': 'JPY',    # Japanese Yen
    'Rs': 'INR',
    'CNH': 'CNY',
    '€': 'EUR',    # Euro
    '¥': 'JPY',    # Alternative Yen symbol
    '£': 'GBP',    # British Pound
    'A$': 'AUD',   # Australian Dollar
    'C$': 'CAD',   # Canadian Dollar
    'HK$': 'HKD',  # Hong Kong Dollar
}

def standardize_currency(currency):
    if pd.isna(currency):
        return np.nan
    if currency in symbol_mapping:
        return symbol_mapping[currency]
    if currency == '':
        return ''
    try:
        if pycountry.currencies.get(alpha_3=currency):
            return currency
    except AttributeError:
        pass
    return currency

def clean_total_net_assets(value):
    if pd.isna(value):
        return np.nan, np.nan
    value = re.sub(r'\basof\b.*', '', value, flags=re.IGNORECASE).strip()
    match = re.match(r'([^0-9\s]+)?\s*([0-9.,]+)\s*([kKmMbB]?)', value)
    if not match:
        return np.nan, np.nan
    currency, num_str, unit = match.groups()
    currency = currency if currency else ''
    num = float(num_str.replace(',', ''))
    unit = unit.lower() if unit else ''
    if unit == 'k':
        num *= 10**3
    elif unit == 'm':
        num *= 10**6
    elif unit == 'b':
        num *= 10**9
    elif unit == 't':
        num *= 10**12
    return num, currency

def get_exchange_rates(currencies, to_currency='USD'):
    rates = {}
    valid_currencies = []
    for c in currencies:
        if pd.notna(c) and pycountry.currencies.get(alpha_3=c) is not None:
            valid_currencies.append(c)
    if not valid_currencies:
        return rates
    try:
        url = f"https://open.er-api.com/v6/latest/{to_currency}"
        response = requests.get(url)
        data = response.json()
        if 'rates' in data:
            for currency in valid_currencies:
                if currency == 'USD':
                    rates[currency] = 1.0
                elif currency in data['rates']:
                    rates[currency] = 1 / data['rates'][currency] if data['rates'][currency] != 0 else np.nan
            # print(f"Fetched rates: {rates}")
            return rates
        else:
            print(f"Error fetching rates: {data.get('error', 'Unknown error')}")
    except Exception as e:
        print(f"Exchange rate fetch failed: {e}")
    return rates

def convert_to_usd(row, rates):
    if pd.isna(row['profile_cap']) or pd.isna(row['profile_cap_currency']):
        return np.nan
    currency = row['profile_cap_currency']
    if currency in rates:
        return row['profile_cap'] * rates[currency]
    print(f"No rate available for {currency}")
    return np.nan

contracts_df[['profile_cap', 'profile_cap_currency']] = contracts_df['profile_TotalNetAssets'].apply(lambda x: pd.Series(clean_total_net_assets(x)))
contracts_df['profile_cap_currency'] = contracts_df['profile_cap_currency'].apply(standardize_currency)
contracts_df['profile_cap_currency'] = np.where(contracts_df['profile_cap_currency'] == '', contracts_df['currency'], contracts_df['profile_cap_currency'])
contracts_df['profile_cap_currency'] = contracts_df['profile_cap_currency'].apply(lambda x: x if (pd.isna(x) or pycountry.currencies.get(alpha_3=x) or x == '') else np.nan)

exchange_rates = get_exchange_rates(contracts_df['profile_cap_currency'].unique())
contracts_df['profile_cap_usd'] = contracts_df.apply(lambda row: convert_to_usd(row, exchange_rates),axis=1)
contracts_df = contracts_df.drop(columns=['profile_TotalNetAssets', 'profile_cap', 'profile_cap_currency'], axis=1, errors='ignore')

# TER
contracts_df['profile_TotalExpenseRatio'] = contracts_df['profile_TotalExpenseRatio'].replace('', np.nan).astype(float)

In [ ]:
# Domicile dummies
if 'profile_Domicile' in contracts_df.columns:
    dummies = pd.get_dummies(contracts_df['profile_Domicile'], prefix='domicile').astype(int)
    contracts_df = pd.concat([contracts_df, dummies], axis=1)
    contracts_df.drop('profile_Domicile', axis=1, inplace=True)

# Market cap dummies
size_map = {
        'Small-cap': 'small',
        'Mid-cap': 'mid',
        'BroadMarket': 'multi',
        'Large-cap': 'large',
    }
contracts_df['profile_MarketCapFocus'] = contracts_df['profile_MarketCapFocus'].map(size_map)

dummies = pd.get_dummies(contracts_df['profile_MarketCapFocus'], dtype=int)
contracts_df = pd.concat([contracts_df, dummies], axis=1)

# Define mapping from size to related style columns
style_groups = {
    'small': ['style_small-core', 'style_small-growth', 'style_small-value'],
    'mid': ['style_mid-core', 'style_mid-growth', 'style_mid-value'],
    'large': ['style_large-core', 'style_large-growth', 'style_large-value'],
    'multi': ['style_multi-core', 'style_multi-growth', 'style_multi-value']
}

# Update each size column by OR-ing with the style columns
for size, cols in style_groups.items():
    contracts_df[size] = contracts_df[size] | contracts_df[cols].any(axis=1).astype(int)

del dummies, size_map, style_groups

In [ ]:
# Search exchange verification
contracts_df['search_exchange'] = contracts_df['search_exchange'].str.extract(r'\(([^()]*)\)', expand=False)
contracts_df['validExchanges'] = contracts_df['validExchanges'].apply(lambda x: x.split(','))

def validate_search_exchange(row):
    # if pd.isna(row['search_exchange']):
    #     return np.nan
    return 1 if row['search_exchange'] in row['validExchanges'] else 0

contracts_df['valid_search_exchange'] = contracts_df.apply(validate_search_exchange, axis=1)
contracts_df = contracts_df.drop(columns=['search_symbol', 'search_exchange', 'validExchanges'], axis=1, errors='ignore')


In [ ]:
# Final cleaning
fill_dict = {
    'holding_types_Equity': 0,
    'holding_types_FixedIncome': 0,
    'holding_types_Cash': 0,
    'holding_types_Other': 1,
    'holding_types_variety': 1,
}
contracts_df = contracts_df.fillna(fill_dict)

# rating encoding
rating_map = {
    'AAA': 6,
    'AA': 5,
    'A': 4,
    'BBB': 3,
    'BB': 2,
    'B': 1,
}
contracts_df['fundamentals_AverageQuality'] = contracts_df['fundamentals_AverageQuality'].map(rating_map)

# # bool rename
# bool_map = {
#     True: 1,
#     False: 0,
#     np.nan: 0,
# }
# bool_cols = [col for col in contracts_df.columns if col.startswith('style_')] + ['tradable']
# for col in bool_cols:
#     contracts_df[col] = contracts_df[col].map(bool_map)

contracts_df.loc[contracts_df['stockType'] == 'ETC', 'industries_BasicMaterials'] = 1.0

# Remove unnecessary qual or empty columns, only keep key identifiers
qual_cols = ['primaryExchange', 'stockType', 'date_scraped', 'exchange_bug', 'exact_search', 'search_symbol', 'profile_MarketCapFocus', 'profile_MarketGeoFocus', 'profile_BenchmarkIndex', 'profile_FundCategory', 'dividends_PayoutRatio']
contracts_df = contracts_df.drop(columns=qual_cols, axis=1, errors='ignore')

# Clean remaining numerical columns
identifier_cols = ['symbol', 'conId', 'longName', 'isin', 'exchange', 'currency', 'profile_MarketCapFocus']
for col in [c for c in contracts_df.columns if c not in identifier_cols]:
    temp_type = contracts_df[col].dtype
    if temp_type == 'object':
        contracts_df[col] = contracts_df[col].apply(lambda x: np.nan if isinstance(x, str) else x)

In [ ]:
# Data Preparation & Initial Setup:
stylebox_cols = [col for col in contracts_df.columns if col.startswith('style_')]

value_metric_cols = ['fundamentals_Price/Book', 'fundamentals_Price/Sales',
                     'fundamentals_Price/Cash', 'fundamentals_Price/Earnings']
                     # Optional: 'fundamentals_LTDebt/Shareholders', 'fundamentals_TotalDebt/TotalEquity'
growth_metric_cols = ['fundamentals_EPSGrowth-1yr', 'fundamentals_EPS_growth_3yr',
                      'fundamentals_EPS_growth_5yr']
                      # Optional: 'fundamentals_ReturnonAssets1Yr', 'fundamentals_SalestoTotalAssets'

bond_fundamental_cols = ['fundamentals_AverageCoupon', 'fundamentals_AverageQuality', 'fundamentals_YieldtoMaturity', 'fundamentals_NominalMaturity', 'fundamentals_EffectiveMaturity']
equity_fundamental_cols = [col for col in contracts_df.columns if col.startswith('fundamentals_') if col not in bond_fundamental_cols]

numerical_cols = [col for col in contracts_df.columns if contracts_df[col].dtype in [np.int64, np.float64] and col not in ['conId']]

In [ ]:
# Split by asset class
bond_mask = contracts_df[bond_fundamental_cols].notna().any(axis=1)
equity_mask = contracts_df[equity_fundamental_cols].notna().any(axis=1)
contracts_df['bond'] = contracts_df['bond'] | bond_mask
contracts_df['other'] = ~(equity_mask | contracts_df['bond'])
contracts_df['other'] = contracts_df['other'] | (equity_mask & contracts_df['bond'])
contracts_df['equity'] = ~(contracts_df['bond'] | contracts_df['other'])

In [ ]:
contracts_df[contracts_df.equity][[col for col in contracts_df if col.startswith('holding_types')]]

In [ ]:
contracts_df[contracts_df.bond][[col for col in contracts_df if col.startswith('holding_types')]]

In [ ]:
contracts_df['holding_types_other'] = contracts_df['holding_types_Cash'] + contracts_df['holding_types_Other']

bond_threshold = contracts_df[contracts_df.bond]['holding_types_FixedIncome'].median() - contracts_df[contracts_df.bond]['holding_types_FixedIncome'].std()
equity_threshold = contracts_df[contracts_df.equity]['holding_types_Equity'].median() - contracts_df[contracts_df.equity]['holding_types_Equity'].std()
other_threshold = contracts_df[contracts_df.other]['holding_types_other'].median() - contracts_df[contracts_df.other]['holding_types_other'].std()

np.array([bond_threshold, equity_threshold, other_threshold]).mean()

In [ ]:
display(contracts_df[contracts_df.bond]['holding_types_other'].describe())
display(contracts_df[contracts_df.bond]['holding_types_Equity'].describe())
display(contracts_df[contracts_df.bond]['holding_types_FixedIncome'].describe())

In [ ]:
contracts_df['holding_types_other'] = contracts_df['holding_types_Cash'] + contracts_df['holding_types_Other']
contracts_df = contracts_df.rename(columns={'holding_types_FixedIncome': 'holding_types_bond',
                                            'holding_types_Equity': 'holding_types_equity'})

holding_types = {'bond', 'equity', 'other'}
lower_thresholds = []
for holding_type in holding_types:
    for other_type in holding_types - {holding_type}:
        median = contracts_df[contracts_df[holding_type]][f'holding_types_{other_type}'].median()
        std = contracts_df[contracts_df[holding_type]][f'holding_types_{other_type}'].std()
        lower_thresholds.append(median + std)
lower_threshold = np.array(lower_thresholds).mean()
lower_threshold
# bond_threshold = contracts_df[contracts_df.bond]['holding_types_FixedIncome'].mean()
# equity_threshold = contracts_df[contracts_df.equity]['holding_types_Equity'].mean()
# other_threshold = contracts_df[contracts_df.other]['holding_types_other'].mean()


In [ ]:
contracts_df['holding_types_other'] = contracts_df['holding_types_Cash'] + contracts_df['holding_types_Other']

bond_mean = contracts_df[contracts_df.bond]['holding_types_FixedIncome'].mean()
equity_mean = contracts_df[contracts_df.equity]['holding_types_Equity'].mean()
other_mean = contracts_df[contracts_df.other]['holding_types_other'].mean()

np.array([bond_mean, equity_mean, other_mean]).mean()

In [ ]:
row = contracts_df.iloc[0]

row[['holding_types_bond', 'holding_types_equity', 'holding_types_other']].sort_values()#.tail(1)

In [ ]:
import pandas as pd

contracts_df['holding_types_other'] = contracts_df['holding_types_Cash'] + contracts_df['holding_types_Other']
contracts_df = contracts_df.rename(columns={'holding_types_FixedIncome': 'holding_types_bond', 'holding_types_Equity': 'holding_types_equity'})

bond_threshold = contracts_df[contracts_df.bond]['holding_types_bond'].median() - contracts_df[contracts_df.bond]['holding_types_bond'].std()
equity_threshold = contracts_df[contracts_df.equity]['holding_types_equity'].median() - contracts_df[contracts_df.equity]['holding_types_equity'].std()
other_threshold = contracts_df[contracts_df.other]['holding_types_other'].median() - contracts_df[contracts_df.other]['holding_types_other'].std()
upper_threshold = np.array([bond_threshold, equity_threshold, other_threshold]).mean()


holding_types = {'bond', 'equity', 'other'}
lower_thresholds = []
for holding_type in holding_types:
    for other_type in holding_types - {holding_type}:
        median = contracts_df[contracts_df[holding_type]][f'holding_types_{other_type}'].median()
        std = contracts_df[contracts_df[holding_type]][f'holding_types_{other_type}'].std()
        lower_thresholds.append(median + std)
lower_threshold = np.array(lower_thresholds).mean()


def refine_classification(row):
    bond = row['holding_types_bond']
    equity = row['holding_types_equity']
    other = row['holding_types_other']
    max_holding = row[['holding_types_bond', 'holding_types_equity', 'holding_types_other']].sort_values().tail(1)
    
    if bond > upper_threshold:
        return pd.Series([True, False, False], index=['bond', 'equity', 'other'])
    elif equity > upper_threshold:
        return pd.Series([False, True, False], index=['bond', 'equity', 'other'])
    elif other > upper_threshold:
        return pd.Series([False, False, True], index=['bond', 'equity', 'other'])
    
    elif bond < lower_threshold or equity < lower_threshold:
        # return True for column corresponding to max_holding
    else:
        return pd.Series([False, False, True], index=['bond', 'equity', 'other'])

contracts_df[['bond', 'equity', 'other']] = contracts_df.apply(refine_classification, axis=1)
contracts_df

In [ ]:
import pandas as pd

contracts_df = contracts_df.rename(columns={'other': 'other_and_mixed'})
contracts_df['holding_types_other'] = contracts_df['holding_types_Cash'] + contracts_df['holding_types_Other']

bond_threshold = contracts_df[contracts_df['bond']]['holding_types_FixedIncome'].mean()
equity_threshold = contracts_df[contracts_df['equity']]['holding_types_Equity'].mean()

def refine_classification(row):
    fixed_income = row['holding_types_FixedIncome']
    equity = row['holding_types_Equity']
    other = row['holding_types_other']
    
    max_holding = max(fixed_income, equity, other)
    
    if fixed_income == max_holding and fixed_income > bond_threshold:
        return pd.Series([True, False, False], index=['bond', 'equity', 'other_and_mixed'])
    # Classify as equity if Equity exceeds threshold and is dominant
    elif equity == max_holding and equity > equity_threshold:
        return pd.Series([False, True, False], index=['bond', 'equity', 'other_and_mixed'])
    # Otherwise, classify as other_and_mixed (mixed or non-dominant)
    else:
        return pd.Series([False, False, True], index=['bond', 'equity', 'other_and_mixed'])

# Step 5: Apply the refinement
contracts_df[['bond', 'equity', 'other_and_mixed']] = contracts_df.apply(refine_classification, axis=1)

# Step 6: Validate holding_types sum
contracts_df['holding_sum'] = (contracts_df['holding_types_FixedIncome'] + 
                              contracts_df['holding_types_Equity'] + 
                              contracts_df['holding_types_other'])
# Warn if sums deviate significantly from 1
invalid_sums = contracts_df[(contracts_df['holding_sum'] < 0.9) | (contracts_df['holding_sum'] > 1.1)]
if not invalid_sums.empty:
    print(f"Warning: {len(invalid_sums)} rows have holding_types sums outside [0.9, 1.1].")

# Step 7: Drop temporary column
contracts_df = contracts_df.drop(columns=['holding_sum'])

In [ ]:
# Impute fundamental values
from sklearn.impute import KNNImputer

# Equity Fundamentals
all_equity_cols = [col for col in numerical_cols if col not in bond_fundamental_cols]
# equity_mask = contracts_df[equity_fundamental_cols].notna().any(axis=1)
temp_df = contracts_df.loc[equity_mask, [col for col in numerical_cols if col not in bond_fundamental_cols]].copy()

imputer = KNNImputer(n_neighbors=10)
imputed_values = imputer.fit_transform(temp_df)

imputed_df = pd.DataFrame(imputed_values, columns=all_equity_cols, index=temp_df.index)
contracts_df.loc[equity_mask, all_equity_cols] = imputed_df

# Bond Fundamentals
all_bond_cols = [col for col in numerical_cols if col not in equity_fundamental_cols]
# bond_mask = contracts_df[bond_fundamental_cols].notna().any(axis=1)
temp_df = contracts_df.loc[bond_mask, all_bond_cols].copy()

imputer = KNNImputer(n_neighbors=10)
imputed_values = imputer.fit_transform(temp_df)

imputed_df = pd.DataFrame(imputed_values, columns=all_bond_cols, index=temp_df.index)
contracts_df.loc[bond_mask, all_bond_cols] = imputed_df

del temp_df, imputed_df, imputed_values

In [ ]:
# Save
# contracts_df.to_csv('data/fundamentals.csv', index=False)
contracts_df#[~(equity_mask + bond_mask)]

# STYLEBOX MODEL

https://gemini.google.com/app/8cafe6813426884b

Okay, I can provide you with a comprehensive Python script to implement the strategy we discussed for filling in missing Morningstar stylebox assignments.

1. Set up a sample DataFrame similar to yours, including missing values.
2. Preprocess the data:
Identify style and fundamental columns.
Impute missing fundamental values using KNNImputer.
3. Determine the "Size" dimension:
Calculate capitalization thresholds from non-missing data.
Categorize ETFs into 'Large', 'Mid', or 'Small' cap.
4. Determine the "Style" dimension (Value, Core, Growth):
Engineer features: Normalize relevant fundamentals and calculate 'Value Score' and 'Growth Score'.
Train a RandomForestClassifier on the subset of data with existing style assignments.
Predict the style for ETFs with missing assignments.
5. Combine Size and Style: Populate the original boolean stylebox columns based on the determined size and predicted style.

This code provides a full pipeline.


## Key things to note and potential refinements:

1. Sample Data: The create_sample_df function generates data that tries to mimic your description. You'll replace this with loading your actual DataFrame.
2. Imputation of Fundamentals: KNNImputer is used. You might experiment with other methods or parameters.
3. Size Thresholds: The logic for small_max_cap and mid_max_cap is based on the distribution in your labeled data. This is a reasonable starting point. In practice, these thresholds are often defined by index providers (like Morningstar, Russell, S&P) and can change over time. If you have access to official breakpoints, using those would be more accurate.
4. Feature Scaling: MinMaxScaler is used for normalizing fundamentals before calculating scores and for model input. This is crucial.
5. Value and Growth Scores: The formulas you provided are implemented. The normalization ensures that for value metrics, lower raw values (like P/B) result in higher components of the Value Score, and for growth metrics, higher raw values (like EPS Growth) result in higher components of the Growth Score.
6. Style Model: RandomForestClassifier is used. It's a good general-purpose classifier. You can experiment with others (Logistic Regression, Gradient Boosting, SVM). class_weight='balanced' is used to handle potential imbalances in Value/Core/Growth categories.
7. "Multi" Categories: This script primarily focuses on assigning to the nine Large/Mid/Small x Value/Core/Growth boxes. The "multi" categories are more complex and would typically require analysis of the fund's holdings diversification across cap tiers. The current code ensures that if a L/M/S style is imputed, the multi-style columns for that ETF are set to 0.
8. Error Handling & Edge Cases: Basic checks are included (e.g., if no labeled data is available). Real-world data can have many more quirks.
9. Evaluation: The script prints a classification report for the style model on a test set. Thoroughly evaluating the quality of imputations is important (e.g., by comparing with a holdout set if you manually classify some, or by checking the logical consistency of results).
10. Column Names: Ensure the column names in the script match exactly with your DataFrame.
11. Fallback for Style Prediction: If the style model cannot be trained (e.g., too few distinct style labels), it currently defaults to predicting 'Core'. You might want a different strategy.
To use this:


## Details
Adjust column name lists (style_cols, equity_fundamental_cols, etc.) if they differ.
Run the script. The imputed_df will be your DataFrame with the stylebox NaNs filled according to the logic.
The final part of the if __name__ == '__main__': block shows how you could add the calculated Value_Score_Display and Growth_Score_Display to the final DataFrame for easier inspection of the model's inputs.
This should give you a very solid starting point!

In [ ]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

def get_size_category_from_style_cols(row, style_cols_map):
    """Extracts size (Large, Mid, Small, Multi) from boolean style columns."""
    for style_col_name, assignments in style_cols_map.items():
        if row[style_col_name] == 1:
            return assignments['size']
    return np.nan

def get_value_growth_category_from_style_cols(row, style_cols_map):
    """Extracts style (Value, Growth, Core) from boolean style columns."""
    for style_col_name, assignments in style_cols_map.items():
        if row[style_col_name] == 1:
            return assignments['style']
    return np.nan

def impute_styleboxes(df_input):
    """
    Fills missing Morningstar stylebox assignments using a hybrid approach.
    """
    df = df_input.copy()

    style_cols = ['style_large-core', 'style_large-growth', 'style_large-value', 
                  'style_mid-core', 'style_mid-growth', 'style_mid-value', 
                  'style_small-core', 'style_small-growth', 'style_small-value',
                  'style_multi-core', 'style_multi-growth', 'style_multi-value']
    
    # Define a mapping for easier interpretation of style columns
    # This helps in extracting existing size/style and later in reconstructing
    style_cols_map = {
        'style_large-value': {'size': 'Large', 'style': 'Value'},
        'style_large-core': {'size': 'Large', 'style': 'Core'},
        'style_large-growth': {'size': 'Large', 'style': 'Growth'},
        'style_mid-value': {'size': 'Mid', 'style': 'Value'},
        'style_mid-core': {'size': 'Mid', 'style': 'Core'},
        'style_mid-growth': {'size': 'Mid', 'style': 'Growth'},
        'style_small-value': {'size': 'Small', 'style': 'Value'},
        'style_small-core': {'size': 'Small', 'style': 'Core'},
        'style_small-growth': {'size': 'Small', 'style': 'Growth'},
        # Multi-cap styles (handling these perfectly is complex, focusing on L/M/S)
        'style_multi-value': {'size': 'Multi', 'style': 'Value'},
        'style_multi-core': {'size': 'Multi', 'style': 'Core'},
        'style_multi-growth': {'size': 'Multi', 'style': 'Growth'},
    }

    # Identify fundamental columns to be used for imputation and feature engineering
    equity_fundamental_cols = [
        'fundamentals_Price/Book', 'fundamentals_Price/Cash', 
        'fundamentals_Price/Earnings', 'fundamentals_Price/Sales',
        'fundamentals_EPS_growth_1yr', 'fundamentals_EPS_growth_3yr',
        'fundamentals_EPS_growth_5yr', 'fundamentals_ReturnonEquity1Yr',
        'fundamentals_LTDebt/Shareholders'
    ]

    # --- 1. Preprocessing: Impute missing fundamental data ---
    imputer = KNNImputer(keep_empty_features=True)
    df[equity_fundamental_cols] = imputer.fit_transform(df[equity_fundamental_cols])

    equity_fundamental_cols += ['profile_cap_usd']

    # Identify rows where stylebox information is missing
    df['style_is_missing'] = df[style_cols].isnull().all(axis=1)
    df_labeled = df[~df['style_is_missing']].copy()
    df_to_impute = df[df['style_is_missing']].copy()

    if df_labeled.empty:
        print("Error: No labeled data available to train models or derive thresholds. Cannot proceed.")
        return df_input

    # --- 2. Determine Size Dimension (Large, Mid, Small) --- ################# WRONG use 'profile_MarketCapFocus' instead
    df_labeled['size_category_actual'] = df_labeled.apply(
        lambda row: get_size_category_from_style_cols(row, style_cols_map), axis=1
    )
    
    # Define capitalization thresholds based on labeled data (excluding 'Multi')
    cap_thresholds = {}
    for size_cat in ['Small', 'Mid', 'Large']:
        caps = df_labeled[df_labeled['size_category_actual'] == size_cat]['profile_cap_usd']
        if not caps.empty:
            cap_thresholds[f'{size_cat.lower()}_cap_min'] = caps.min()
            cap_thresholds[f'{size_cat.lower()}_cap_max'] = caps.max()
            cap_thresholds[f'{size_cat.lower()}_cap_median'] = caps.median()

    # Define boundaries (example logic, can be refined)
    small_max_cap = cap_thresholds.get('small_cap_max', df['profile_cap_usd'].quantile(0.33) if not df_labeled.empty else 0)
    mid_max_cap = cap_thresholds.get('mid_cap_max', df['profile_cap_usd'].quantile(0.66) if not df_labeled.empty else 0)

    if not cap_thresholds: # Fallback if no labeled data for some size categories
        print("Warning: Not enough labeled data to define all cap thresholds robustly. Using quantiles as fallback.")
        small_max_cap = df['profile_cap_usd'].quantile(0.33)
        mid_max_cap = df['profile_cap_usd'].quantile(0.66)


    def assign_size_category(cap_usd):
        if cap_usd <= small_max_cap:
            return 'Small'
        elif cap_usd <= mid_max_cap:
            return 'Mid'
        else:
            return 'Large'

    df_to_impute['size_category_imputed'] = df_to_impute['profile_cap_usd'].apply(assign_size_category)
    print("Size dimension determined for missing rows.\n")

    # --- 3. Determine Style Dimension (Value, Core, Growth) using a Model ---
    print("Step 3: Determining style dimension (Value/Core/Growth) via classification model...")
    
    # Feature Engineering for Style Model
    # Normalize relevant fundamental ratios (0 to 1 range)
    value_metric_cols = ['fundamentals_Price/Book', 'fundamentals_Price/Sales', 
                         'fundamentals_Price/Cash', 'fundamentals_Price/Earnings']
    growth_metric_cols = ['fundamentals_EPS_growth_1yr', 'fundamentals_EPS_growth_3yr', 
                          'fundamentals_EPS_growth_5yr']
    
    # Use a copy for scaling to avoid changing original imputed values in df_labeled/df_to_impute directly for this step
    df_scaled_features = df[equity_fundamental_cols].copy()
    scaler = MinMaxScaler()
    df_scaled_features[value_metric_cols + growth_metric_cols] = scaler.fit_transform(
        df_scaled_features[value_metric_cols + growth_metric_cols]
    )

    # Calculate Value Score: lower P/B, P/S etc. is better for value.
    # (1 - Normalized_Metric) makes higher scores better for value if Normalized_Metric is 0 (best) to 1 (worst)
    # Our MinMaxScaler maps lowest raw value to 0 and highest to 1.
    # So, for P/B, low P/B (good for value) -> 0. (1-0)=1. High P/B (bad for value) -> 1. (1-1)=0. This is correct.
    df_scaled_features['Value_Score'] = df_scaled_features[value_metric_cols].apply(
        lambda row: np.mean([1 - val for val in row]), axis=1
    )
    
    # Calculate Growth Score: higher EPS growth is better.
    # Normalized_Metric is 0 (worst) to 1 (best). This is correct.
    df_scaled_features['Growth_Score'] = df_scaled_features[growth_metric_cols].mean(axis=1)

    # Prepare data for style classification model
    # Target variable: 'Value', 'Core', 'Growth'
    df_labeled['style_vg_actual'] = df_labeled.apply(
        lambda row: get_value_growth_category_from_style_cols(row, style_cols_map), axis=1
    )
    
    # Filter out 'Multi' styles for training this specific V/G/C model, and also any rows where style_vg_actual is NaN
    # (e.g. if a multi-cap was the only thing labeled, or if style_cols were all 0 but not NaN)
    df_train_style = df_labeled[
        df_labeled['size_category_actual'].isin(['Large', 'Mid', 'Small']) & 
        df_labeled['style_vg_actual'].notna()
    ].copy()


    if df_train_style.empty or df_train_style['style_vg_actual'].nunique() < 2:
        print("Error: Not enough valid labeled data (L/M/S sizes and distinct V/G/C styles) to train style model. Skipping style imputation.")
        # If style model cannot be trained, we can only fill size, or return as is.
        # For this example, we'll proceed to fill based on size only if style model fails.
        # A more robust solution might involve other fallback logic for style.
        predicted_styles = pd.Series(index=df_to_impute.index, dtype='object').fillna('Core') # Fallback to 'Core'
    else:
        # Features for the model
        # We use the scaled features from df_scaled_features, aligning by index
        feature_columns_model = ['Value_Score', 'Growth_Score'] + value_metric_cols + growth_metric_cols + ['fundamentals_ReturnonEquity1Yr', 'fundamentals_LTDebt/Shareholders']
        
        X = df_scaled_features.loc[df_train_style.index, feature_columns_model]
        y = df_train_style['style_vg_actual']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y if y.nunique() > 1 else None)

        print(f"  Training style model with {len(X_train)} samples.")
        style_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
        style_model.fit(X_train, y_train)

        print("  Style model training complete. Evaluating on test set:")
        y_pred_test = style_model.predict(X_test)
        print(classification_report(y_test, y_pred_test, zero_division=0))

        # Predict style for rows that need imputation
        X_to_predict_style = df_scaled_features.loc[df_to_impute.index, feature_columns_model]
        if not X_to_predict_style.empty:
            predicted_styles = style_model.predict(X_to_predict_style)
            df_to_impute['style_vg_imputed'] = predicted_styles
        else:
            df_to_impute['style_vg_imputed'] = 'Core' # Fallback if no rows to impute style for (should not happen if df_to_impute is not empty)
        print("Style dimension (Value/Core/Growth) predicted for missing rows.\n")


    # --- 4. Combine Size and Style & Populate Original Columns ---
    print("Step 4: Combining imputed size and style to fill original stylebox columns...")
    imputed_count = 0
    for idx, row in df_to_impute.iterrows():
        size_cat = row['size_category_imputed']
        # Use predicted style if available, otherwise fallback (e.g. 'Core')
        style_cat = row.get('style_vg_imputed', 'Core') 

        imputed_flag = False
        for s_col, assignments in style_cols_map.items():
            # Focus on L/M/S for this imputation logic
            if assignments['size'] in ['Large', 'Mid', 'Small']:
                if assignments['size'] == size_cat and assignments['style'] == style_cat:
                    df.loc[idx, s_col] = 1
                    imputed_flag = True
                else:
                    # Ensure other L/M/S style columns for this ETF are 0
                    if df.loc[idx, s_col] != 0 : # Check if it was NaN before
                         df.loc[idx, s_col] = 0 
            else: # For Multi-cap columns, ensure they are 0 if we imputed L/M/S
                if df.loc[idx, s_col] != 0:
                    df.loc[idx, s_col] = 0
        
        if imputed_flag:
            imputed_count += 1
        
        # Mark that this row's style has been processed for imputation
        df.loc[idx, 'style_is_missing'] = False 


    # Fill any remaining NaNs in style columns with 0 (e.g. if they were not part of the imputation logic)
    df[style_cols] = df[style_cols].fillna(0)

    print(f"Imputed stylebox for {imputed_count} ETFs.")
    print("Stylebox imputation process complete.\n")
    
    return df



# Create sample data
raw_df = contracts_df.copy()
print("Original DataFrame (sample):")
print(raw_df.head())
print(f"\nMissing style assignments before imputation: {raw_df[style_cols].isnull().all(axis=1).sum()} rows")
print(f"Missing values in 'fundamentals_Price/Book': {raw_df['fundamentals_Price/Book'].isnull().sum()} rows\n")

# Perform imputation
imputed_df = impute_styleboxes(raw_df)

print("\nDataFrame after stylebox imputation (sample):")
# Show some rows that were originally missing styles
original_missing_indices = raw_df[raw_df[style_cols].isnull().all(axis=1)].index
if not original_missing_indices.empty:
    print(imputed_df.loc[original_missing_indices].head(10)[style_cols + ['profile_cap_usd', 'Value_Score', 'Growth_Score', 'size_category_imputed', 'style_vg_imputed']])
else:
    print("No rows had missing styles in the sample (or all were filled by other means).")

print(f"\nMissing style assignments after imputation: {imputed_df[style_cols].isnull().all(axis=1).sum()} rows")
# Verify that NaNs in style columns are filled (should be 0 or 1)
print(f"NaNs remaining in any style column: {imputed_df[style_cols].isnull().sum().sum()}")

# Check one imputed row in detail
if not original_missing_indices.empty:
    example_imputed_ticker = original_missing_indices[0]
    print(f"\nDetails for an imputed ETF ({example_imputed_ticker}):")
    # Add imputed scores to the final df for inspection if they were calculated
    # The scores are on df_scaled_features, need to merge them back if desired for final output
    # For now, just show the relevant columns from imputed_df
    print(imputed_df.loc[example_imputed_ticker, style_cols + ['profile_cap_usd']])
    # To see Value_Score and Growth_Score, they would need to be added to the main 'df' inside 'impute_styleboxes'
    # or merged back from 'df_scaled_features'.
    # For simplicity in this script, they are intermediate.
    # If you want them in the final output, you'd do:
    # df = df.join(df_scaled_features[['Value_Score', 'Growth_Score']]) inside impute_styleboxes before returning.


# --- Example of how to add scores to the final df for inspection ---
# This part is illustrative and would typically be integrated into the main function
# Re-run feature engineering part to get scores on the final imputed_df (if not already done within the function)

temp_df_for_scores = imputed_df.copy()
# Ensure fundamentals are imputed in this temp_df if they weren't already fully imputed on 'imputed_df'
# (they should be from the KNNImputer step)

equity_fundamental_cols_scores = [
    'profile_cap_usd', 'fundamentals_Price/Book', 'fundamentals_Price/Cash', 
    'fundamentals_Price/Earnings', 'fundamentals_Price/Sales',
    'fundamentals_EPS_growth_1yr', 'fundamentals_EPS_growth_3yr',
    'fundamentals_EPS_growth_5yr', 'fundamentals_ReturnonEquity1Yr',
    'fundamentals_LTDebt/Shareholders'
]
value_metric_cols_scores = ['fundamentals_Price/Book', 'fundamentals_Price/Sales', 
                        'fundamentals_Price/Cash', 'fundamentals_Price/Earnings']
growth_metric_cols_scores = ['fundamentals_EPS_growth_1yr', 'fundamentals_EPS_growth_3yr', 
                        'fundamentals_EPS_growth_5yr']

# Make sure all necessary columns exist and are numeric
for col in equity_fundamental_cols_scores:
    if col not in temp_df_for_scores.columns:
        print(f"Warning: Column {col} not found for score calculation display. Filling with 0.")
        temp_df_for_scores[col] = 0 
    temp_df_for_scores[col] = pd.to_numeric(temp_df_for_scores[col], errors='coerce').fillna(0)


scaler_display = MinMaxScaler()
temp_df_for_scores[value_metric_cols_scores + growth_metric_cols_scores] = scaler_display.fit_transform(
    temp_df_for_scores[value_metric_cols_scores + growth_metric_cols_scores]
)
temp_df_for_scores['Value_Score_Display'] = temp_df_for_scores[value_metric_cols_scores].apply(
    lambda row: np.mean([1 - val for val in row]), axis=1
)
temp_df_for_scores['Growth_Score_Display'] = temp_df_for_scores[growth_metric_cols_scores].mean(axis=1)

if not original_missing_indices.empty:
    print("\nImputed ETF with calculated scores (for display):")
    print(temp_df_for_scores.loc[original_missing_indices[0], style_cols + ['profile_cap_usd', 'Value_Score_Display', 'Growth_Score_Display']])



In [ ]:
contracts_df#.to_csv('data/fundamentals.csv', index=False)

# EXTRA

In [ ]:
# Reduce to numerical columns
basic_classification = [col for col in original_columns if col not in columns_to_explode]
bond_fundamentals = ['fundamentals_AverageQuality', 'fundamentals_NominalMaturity', 'fundamentals_EffectiveMaturity', 'fundamentals_AverageCoupon', 'fundamentals_YieldtoMaturity']
profile_classification = ['profile_Domicile', 'profile_MarketGeoFocus', 'profile_BenchmarkIndex', 'profile_FundCategory', 'profile_TotalExpenseRatio', 'profile_TotalNetAssets', 'profile_cap', 'profile_cap_currency', 'profile_MarketCapFocus']

classification_columns = basic_classification + bond_fundamentals + profile_classification
data_cols = contracts_df.columns[~contracts_df.columns.isin(classification_columns)]

data = contracts_df[~contracts_df['fundamentals_Price/Book'].isna()].copy()
data = data[~data['fundamentals_LTDebt/Shareholders'].isna()]
data = data[~data['style_large-core'].isna()]

fundamental_columns = [full_column for full_column in contracts_df.columns if full_column.startswith('fundamentals') and full_column not in bond_fundamentals]
for col in fundamental_columns + ['profile_cap_usd']:
    print(col)
    data[col] = (data[col] - data[col].mean()) / data[col].std()

data[data_cols]

In [ ]:
# Graph correlations
import seaborn as sns
import matplotlib.pyplot as plt

# drop columns with missing values
corr_df = data[data_cols].corr()
corr_df.dropna(axis=1, how='all', inplace=True)
corr_df.dropna(axis=0, how='all', inplace=True)
data_cols = corr_df.columns

plt.figure(figsize=(50, 50))
sns.heatmap(corr_df, cmap='coolwarm')
plt.show()

---
### Factor definition
---

In [ ]:
'''
# Academic factors
Market beta
SMB
HML
RMW


# Variety factors
approaches 1 => little variety
approaches 0 => a lot of variety
sum of country**2
sum of currency**2
sum of industry**2
sum of top10**2

Check if factors are orthogonal
'''

'''
# Variety factors
approaches 1 => little variety
approaches 0 => a lot of variety
sum of country**2
sum of currency**2
sum of industry**2
sum of top10**
'''

In [ ]:
# Fill in style columns
fundamental_columns = [full_column for full_column in contracts_df.columns if full_column.startswith('fundamentals') and full_column not in bond_fundamentals]

value_columns = ['fundamentals_Price/Book',  'fundamentals_Price/Cash', 'fundamentals_Price/Earnings', 'fundamentals_Price/Sales']#, 'fundamentals_LTDebt/Shareholders', 'fundamentals_TotalDebt/TotalCapital', 'fundamentals_TotalDebt/TotalEquity']#,  'fundamentals_TotalAssets/TotalEquity']
growth_columns = ['fundamentals_EPSGrowth-1yr', 'fundamentals_EPS_growth_3yr', 'fundamentals_EPS_growth_5yr']#, 'fundamentals_ReturnonAssets', 'fundamentals_SalestoTotalAssets']
output_columns = [full_column for full_column in contracts_df.columns if full_column.startswith('style')]

'''
growth score = 2 * [ N_P/B + N_P/E + N_P/Cash + N_P/Sales + N_EPS_growth_1yr + N_EPS_growth_3yr + N_EPS_growth_5yr + 
              N_ReturnonAssets1Yr + N_ReturnonAssets3Yr + N_ReturnonCapital + N_ReturnonCapital3Yr + 
              N_ReturnonEquity1Yr + N_ReturnonEquity3Yr + N_ReturnonInvestment1Yr + N_ReturnonInvestment3Yr + 
              N_SalestoTotalAssets + N_EBITtoInterest + N_RelativeStrength + 
              (1 - N_LTDebt/ShareholdersEquity) + (1 - N_TotalAssets/TotalEquity) + 
              (1 - N_TotalDebt/TotalCapital) + (1 - N_TotalDebt/TotalEquity) ] / 22 - 1

Extreme Growth: If all growth indicators ≈ 1 and value indicators ≈ 0, then S = [18*1 + 4*1]/22 = 1, score = 2*1 - 1 = 1.
Extreme Value: If all growth indicators ≈ 0 and value indicators ≈ 1, then S = [18*0 + 4*0]/22 = 0, score = 2*0 - 1 = -1.
Neutral: If all ≈ 0.5, then S = [18*0.5 + 4*0.5]/22 = 0.5, score = 2*0.5 - 1 = 0.


Step 4: Proposed Refined Model
Balancing your suggestions with practicality and Morningstar’s framework, I recommend:
Select Key Metrics: Use only the most relevant IBKR metrics.
Equal Weighting Within Categories: Follow Morningstar’s approach for simplicity and grounding.
Score Calculation: Compute a value-growth spectrum from -1 to 1.
Refined Model
Value Score = mean((1 - N_P/B) + (1 - N_P/Sales) + (1 - N_P/Cash) + (1 - N_P/E)) # Possibly add: LTDebt/ShareholdersEquity, TotalDebt/Equity
Growth Score = mean(N_EPS_growth_1yr + N_EPS_growth_3yr + N_EPS_growth_5yr) # Possibly add: ReturnonAssets, SalestoTotalAssets

Why This Works
Relevance: Uses metrics tied to Morningstar’s historical measures and value investing principles.
Simplicity: Equal weighting avoids overcomplication while mirroring industry practice.
No Additional Standardization: Normalization suffices for comparability.
Flexibility: Captures the spectrum effectively with available data.

'''


---
### Clustering analysis
---

In [ ]:
# distance correlation
import dcor

training_array = data[data_cols].values # Convert training matrix to numpy array
symbol_list = data[data_cols].columns.tolist()
num_symbols = len(symbol_list)
corr_matrix = np.zeros((num_symbols, num_symbols)) # Pre-allocate numpy array for correlation
cov_matrix = np.zeros((num_symbols, num_symbols))  # Pre-allocate numpy array for covariance

for i, sym_i in tqdm(enumerate(symbol_list), total=num_symbols, desc=f"Calculating distance stats sqr"):
    for j, sym_j in enumerate(symbol_list):
        if i <= j:  # Compute only for upper triangle (including diagonal)
            stats = dcor.distance_stats(training_array[:, i], training_array[:, j])
            corr_value = stats.correlation_xy
            cov_value = stats.covariance_xy

            corr_matrix[i, j] = corr_value
            corr_matrix[j, i] = corr_value  # Fill symmetric value

            cov_matrix[i, j] = cov_value
            cov_matrix[j, i] = cov_value  # Fill symmetric value

corr_df = pd.DataFrame(corr_matrix, index=symbol_list, columns=symbol_list) # Convert numpy array back to df for output
cov_df = pd.DataFrame(cov_matrix, index=symbol_list, columns=symbol_list)   # Convert numpy array back to df for output


# drop columns with missing values
corr_df = data[data_cols].corr()#.values
corr_df = corr_df.dropna(axis=1, how='all')
corr_df.dropna(axis=0, how='all', inplace=True)
corr_df.shape

In [ ]:
# Create distance matrix
symbol_list = corr_df.columns

symbol2index = dict(zip(corr_df.columns, corr_df.index))
index2symbol = dict(zip(corr_df.index, corr_df.columns))
corr_df.rename(columns=symbol2index, inplace=True)
# cov_df.rename(columns=symbol2index, inplace=True)

distance_matrix = (1 - corr_df).to_numpy()
np.fill_diagonal(distance_matrix, 0)

In [ ]:
# Thresholds / cluster_num graphs
methods = ['single', 'ward', 'average', 'complete', 'weighted', 'centroid', 'median']
methods = ['ward']
for method in methods:
    linked = sch.linkage(squareform(distance_matrix), method=method)
    
    num_clusters = range(len(corr_df), 1, -1)
    thresholds = linked[:, 2]

    plt.figure(figsize=(12, 6))
    plt.plot(num_clusters, thresholds, marker='o')
    plt.title(f"Threshold/Num ({method})")
    plt.xlabel('Number of Clusters')
    plt.ylabel('Threshold (Distance)')
    plt.grid(True)
    plt.show()


In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
import scipy.cluster.hierarchy as sch
from scipy.spatial.distance import squareform

# Silhouettes and dendrograms
def product(row):
    product = 1
    for value in row.values():
        product *= value
    return product

methods = ['single', 'ward', 'average', 'complete', 'weighted', 'centroid', 'median']
methods = ['ward']
for method in methods:

    ks = []
    scores = []
    counts = []
    for k in range(2, min(len(distance_matrix), 9)):
        clusters = AgglomerativeClustering(n_clusters=k, linkage=method).fit_predict(distance_matrix)
        score = silhouette_score(distance_matrix, clusters, metric='precomputed')
        ks.append(k)
        scores.append(score)
        unique_clusters, label_counts = np.unique(clusters, return_counts=True)
        label_counts_dict = dict(zip(unique_clusters, label_counts))
        counts.append(label_counts_dict)

    silhouettes = pd.DataFrame({
        'k': ks,
        'score': scores,
        'counts': counts
    })
    silhouettes['combitions'] = silhouettes['counts'].apply(product)
    silhouettes = silhouettes.sort_values(by='score', ascending=False)
    best_k = silhouettes.k.iloc[0]
    display(silhouettes)

    linked = sch.linkage(squareform(distance_matrix), method=method)
    plt.figure(figsize=(40, 15))
    sch.dendrogram(linked, labels=corr_df.index, leaf_rotation=90)
    plt.title(f"Method {method}")
    plt.show()

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
import scipy.cluster.hierarchy as sch
from scipy.spatial.distance import squareform

# Silhouettes and dendrograms
def product(row):
    product = 1
    for value in row.values():
        product *= value
    return product

methods = ['single', 'ward', 'average', 'complete', 'weighted', 'centroid', 'median']
methods = ['ward', 'complete']
for method in methods:

    ks = []
    scores = []
    counts = []
    for k in range(2, min(len(distance_matrix), 9)):
        clusters = AgglomerativeClustering(n_clusters=k, linkage=method).fit_predict(distance_matrix)
        score = silhouette_score(distance_matrix, clusters, metric='precomputed')
        ks.append(k)
        scores.append(score)
        unique_clusters, label_counts = np.unique(clusters, return_counts=True)
        label_counts_dict = dict(zip(unique_clusters, label_counts))
        counts.append(label_counts_dict)

    silhouettes = pd.DataFrame({
        'k': ks,
        'score': scores,
        'counts': counts
    })
    silhouettes['combitions'] = silhouettes['counts'].apply(product)
    silhouettes = silhouettes.sort_values(by='score', ascending=False)
    best_k = silhouettes.k.iloc[0]
    display(silhouettes)

    linked = sch.linkage(squareform(distance_matrix), method=method)
    plt.figure(figsize=(40, 15))
    sch.dendrogram(linked, labels=corr_df.index, leaf_rotation=90)
    plt.title(f"Method {method}")
    plt.show()